In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 54.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c39ba8656f1b23ceac26bc9206d98152f590aef48de2308391ca3d2cac8ca17b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-20 14:13:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2022-12-20 14:13:45 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()
     

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...|2015-08-31 00:00:00|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     E

In [5]:
# filter DataFrame for total_votes above or equal to 20

new_df = df.filter(df.total_votes >= 20)
new_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...|2015-08-18 00:00:00|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERR

In [6]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
helpful_votes_df = new_df.filter((new_df.helpful_votes / new_df.total_votes) >= 0.5)
helpful_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...|2015-08-18 00:00:00|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERR

In [7]:
# Create paid vine DataFrame
paid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...|2014-01-13 00:00:00|
|         US|   50929595|R1RZ4JZO8DAP7H|B00DSICT1Y|     427004417|uComfy

In [8]:
# Create unpaid vine DataFrame
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...|2015-08-18 00:00:00|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERR

In [9]:
# Total paid reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews  

3

In [10]:
# paid 5-star reviews
five_star_review = paid_vine_df.filter(paid_vine_df.star_rating ==5).count()
five_star_review

2

In [12]:
# paid 5-star reviews percentage
five_star_percent = (five_star_review / total_paid_reviews) * 100
five_star_percent

66.66666666666666

In [13]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

3094

In [14]:
# unpaid 5-star reviews
unpaid_five_star_review = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
unpaid_five_star_review

1704

In [17]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_review / total_unpaid_reviews) * 100
unpaid_five_star_percent

55.07433742727861